In [13]:
#imports
import torch
import torch.nn as nn # All Neural Network models
import torch.optim as optim # All Optimization
from torch.utils.data import DataLoader # Easier Dataset Managment 
import torchvision.transforms as transforms # Transformations we can do
import torchvision.datasets as datasets # to import the needed dataset
import torchvision 
from peft import LoraConfig, TaskType
from peft import get_peft_model

In [19]:
# Set Device & Hyperparameters & Model Loading
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 32
epochs = 20

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the model
for param in model.parameters():
    param.requires_grad = False
    
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 100) # CIFAR100 has 100 classes
lora_config = LoraConfig(
    r=16,
    target_modules=["fc"],
    lora_alpha=32,
    lora_dropout=0.05
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

_ = lora_model.to(device)

trainable params: 9,792 || all params: 11,237,604 || trainable%: 0.08713601226738368


In [20]:
# Transformation 
# Define the standard ImageNet transforms
transform = transforms.Compose([
    transforms.Resize(256),              # Resize the image to 256x256 pixels
    transforms.CenterCrop(224),          # Crop the image to 224x224 pixels at the center
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # Normalize pixel values to match the ImageNet distribution
                         std=[0.229, 0.224, 0.225]),
])

In [21]:
# Load Training Set
trainset = datasets.CIFAR100(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

# Load Test Set
testset = datasets.CIFAR100(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lora_model.parameters(), lr=0.001, momentum=0.9)
torch.cuda.empty_cache()

In [22]:
# Training The Model
for epoch in range(epochs):
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = lora_model(inputs)
        loss = criterion(
            outputs, labels
        )
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if i % (batch_size*2) == ((batch_size*2) - 1):
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,    64] loss: 3.075
[1,   128] loss: 3.012
[1,   192] loss: 2.993
[1,   256] loss: 2.990
[1,   320] loss: 2.975
[1,   384] loss: 2.939
[1,   448] loss: 2.904
[1,   512] loss: 2.864
[1,   576] loss: 2.802
[1,   640] loss: 2.728
[1,   704] loss: 2.662
[1,   768] loss: 2.587
[1,   832] loss: 2.526
[1,   896] loss: 2.452
[1,   960] loss: 2.402
[1,  1024] loss: 2.326
[1,  1088] loss: 2.281
[1,  1152] loss: 2.235
[1,  1216] loss: 2.172
[1,  1280] loss: 2.094
[1,  1344] loss: 2.088
[1,  1408] loss: 2.037
[1,  1472] loss: 2.023
[1,  1536] loss: 1.987
[2,    64] loss: 1.932
[2,   128] loss: 1.903
[2,   192] loss: 1.909
[2,   256] loss: 1.861
[2,   320] loss: 1.826
[2,   384] loss: 1.834
[2,   448] loss: 1.832
[2,   512] loss: 1.776
[2,   576] loss: 1.772
[2,   640] loss: 1.742
[2,   704] loss: 1.793
[2,   768] loss: 1.716
[2,   832] loss: 1.729
[2,   896] loss: 1.709
[2,   960] loss: 1.716
[2,  1024] loss: 1.654
[2,  1088] loss: 1.703
[2,  1152] loss: 1.685
[2,  1216] loss: 1.658
[2,  1280] 

In [23]:
# Testing The Model    
_ = lora_model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = lora_model(images)
        _, predicted = torch.max(
            outputs.data, 1
        )
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy : {100 * correct / total} %')

Accuracy : 43.11 %
